In [1]:
"""
/**
 * Implementation of the Selfridge Conway envy free cake-cutting procedure on a 1-dimensional cake
 *
 * @author Thomas Gibson
 * @date 30/04/2021
 */
"""



import numpy as np, numpy.random
import random as ran


from Agent import Agent


#This function will generate n values that will sum to exactly 1. 
#This will form an array showing how much an agent values a single atom
#This generating function could be changed to make use of other distributions if preferred
def generatevaluations(n): 
    x = np.random.random(n)
    x /= x.sum()
    return x

#This function will determine an agents value of a piece (vop)
def vop(agent, piece):
    value = 0
    a, b = piece   
    
    #This line is needed as the range function is not inclusive on the top end
    b = b+1
    
    for x in range(a, b):
        value = value + agent.vals[x]
    return value    
    
    
#This function is the first set of cuts done in the procedure.
#Each piece will be roughly equal to a third of the total value, depending on how the atoms align
def initialcuts(agent, numofatoms):
    i = 0
    v1 = 0
    v2 = 0
    while v1 <= (1/3):
        v1 = v1 + agent.vals[i]
        i+=1
    piece1delimiter = i-1
    
    while v2 <= (1/3):
        v2 = v2 + agent.vals[i]
        i+=1
    piece2delimiter = i-1
    
    piece1 = (0, piece1delimiter)
    piece2 = (piece1delimiter+1, piece2delimiter)
    piece3 = (piece2delimiter+1, numofatoms-1)
    
    return (piece1, piece2, piece3)

#This is the function that does the trim, it is a "lefthand" trim.
#This means it scans from the "left" of the cake and to the left of the knife will be the trimming.
#
def lefttrim(agent, piece1, piece1val, piece2, piece2val):
    p1a, p2b = piece1
    i = p1a
    valuep1 = piece1val
    
    
    while i <= p2b:
        valuep1 = valuep1 - agent.vals[i]
        if(valuep1 <= piece2val):
            break
        else:
            i+=1
        
    newpiece1 = (i+1, p2b)
    trimming = (p1a, i)
    
    return (newpiece1, trimming)
    
#This function returns the pieces post trimming with the output pieces in order of preference of the trimmer
#The first piece outputted is the trimmed piece, the last piece is the trimmings
def createtrim(agent, piece1, piece2, piece3):
    piece1val = vop(agent, piece1)
    piece2val = vop(agent, piece2)    
    piece3val = vop(agent, piece3)    
    
    #generate 2d array for the pieces to allow us to sort them
    array = [[piece1, piece1val], [piece2, piece2val], [piece3, piece3val]]
    sortedarray = sorted(array, key=lambda x:x[1], reverse=True)
    
    sortedarray[0][0], trim = lefttrim(agent, sortedarray[0][0], sortedarray[0][1], sortedarray[1][0], sortedarray[1][1])
    
    piece1 = sortedarray[0][0]
    piece2 = sortedarray[1][0]
    piece3 = sortedarray[2][0]
    
    return piece1, piece2, piece3, trim


#The first set of allocations as outlined by selfridge conway procedure.
#The input of the pieces should be the order of preferrence of the trimming agent
#This would be the order of pieces outputted by the createtrim function
def firstallocation(agent1, agent2, agent3, piece1, piece2, piece3):
    #Give agent 3 his most prefered piece
    if ((vop(agent3,piece1) >= vop(agent3, piece2)) and (vop(agent3,piece1) >= vop(agent3, piece3))): 
        agent3.allocation = piece1
        agent3.holdingtrimmed = 1
    elif ((vop(agent3, piece2) >= vop(agent3, piece1)) and (vop(agent3, piece2) >= vop(agent3, piece3))): 
        agent3.allocation = piece2
        agent3.holdingtrimmed = 2 
    else:
        agent3.allocation = piece3

    #Give agent 2 which ever most prefered piece is left (eitehr trimmed piece or piece equal to it)
    if (agent3.allocation == piece1):
        agent2.allocation = piece2
        agent2.holdingtrimmed = 2       
    else:
        agent2.allocation = piece1
        agent2.holdingtrimmed = 1    
    #Give agent 1 the remaining piece
    if ((agent2.allocation == piece2) or (agent3.allocation == piece2)):
        agent1.allocation = piece3
    else:
        agent1.allocation = piece2

        
        

#This function returns which agent should cut the trimmings
def trimmedholdercheck(agent2, agent3):
    if agent2.holdingtrimmed == 1:
        return agent3
    else:
        return agent2

 #This function returns which agent should get first choice of the divided trimmings
def trimmingchoosercheck(agent2, agent3):
    if agent2.holdingtrimmed == 1:
        return agent2
    else:
        return agent3   
    
#This function divides the trimmings into 3 where possible. We must account for when we have less than 3 atoms.
#If there are less than 3 atoms, we might have less than 3 pieces of trimming
#With more atoms this is less likely to occur
def dividetrim(agent, trimmings):
    ta, tb = trimmings
    i = ta
    v1 = 0
    v2 = 0
    while v1 <= ((1/3)*vop(agent, trimmings)):
        v1 = v1 + agent.vals[i]
        i += 1
        if(i > tb):
            break
    trim1delimiter = i-1
    
    
    while v2 <= ((1/3)*vop(agent, trimmings)):
        if(i > tb):
            break
        else:
            v2 = v2 + agent.vals[i]
            i+=1
    trim2delimiter = i-1
    
    
    if(i > tb) and (trim2delimiter == trim1delimiter):
        trim1 = (ta, trim1delimiter)
        trim2 = 0
        trim3 = 0
    elif(i > tb):
        trim1 = (ta, trim1delimiter)
        trim2 = (trim1delimiter+1, trim2delimiter)
        trim3 = 0
    else:
        trim1 = (ta, trim1delimiter)
        trim2 = (trim1delimiter+1, trim2delimiter)
        trim3 = (trim2delimiter+1, tb)
    return (trim1, trim2, trim3)

#This function is what is called to divide the trimming and mades use of the previous function to provide the cuts
#The main purpose here is to determine the agent who will cut the trimmings
def trimcut(agent1, agent2, trimmings):
    #Determine who the cutter agent will be
    cutteragent = trimmedholdercheck(agent1, agent2)
    
    #Divide the trimmings into 3 pieces
    trim1, trim2, trim3 = dividetrim(cutteragent, trimmings)
    
    return trim1, trim2, trim3
#This function will allocate the pieces.
#We must be care to only allocate a piece of the trimming if such a piece exists
def trimallocation(agent1, agent2, agent3, trim1, trim2, trim3):
    #Determine who did the trimming and who hold the trimming piece to determine order
    #We know agent 2 and 3 are these agents we just dont know who is who without checking explicitly
    chooseragent = trimmingchoosercheck(agent2, agent3)
    lastagent  = trimmedholdercheck(agent2, agent3)
    
    #If only one trim piece exists we simply give it to the first choosing agent
    if ((trim2 and trim3) == 0):
        chooseragent.trim = trim1
    
    #If only 2 pieces exist, give first agent free choice and remaining agent to second agent
    elif(trim3 == 0):
        if(vop(chooseragent, trim1) > (vop(chooseragent, trim2))):
            chooseragent.trim = trim1
            agent1.trim = trim2
        else:
            chooseragent.trim = trim2
            agent1.trim = trim1
    else:
        #generate 2d array and sort them in decending value order according to chooser agent
        #All agents get a piece here and select in order of chooser agent, agent1, and lastagent.
        array = [[trim1, vop(chooseragent, trim1)], [trim2, vop(chooseragent, trim2)], [trim3, vop(chooseragent, trim3)]]
        sortedarray = sorted(array, key=lambda x:x[1], reverse=True)
        chooseragent.trim = sortedarray[0][0]
        if (vop(agent1, sortedarray[1][0]) > vop(agent1, sortedarray[2][0])):
            agent1.trim = sortedarray[1][0]
            lastagent.trim = sortedarray[2][0]
        else:
            agent1.trim = sortedarray[2][0]
            lastagent.trim = sortedarray[1][0]           
    

#Sets their total allocation value to be both pieces they could have recieved    
def determineta(agent1, agent2, agent3):
        agent1.totalallocation = [agent1.allocation, agent1.trim]
        agent2.totalallocation = [agent2.allocation, agent2.trim]
        agent3.totalallocation = [agent3.allocation, agent3.trim]
        
#This function checks if an agent is holding a trimmed piece or not        
def trimmingspiece(agent):
    if agent.trim == 0:
        return 0
    else:
        return 1
    
#Print the allocation and relevant infomation in this function
#We also use this to attribute each agent with their own percived values
def printallocation(agent1, agent2, agent3):
    if agent1.totalallocation[1] == 0:
        agent1.totalvalue = vop(agent1, agent1.totalallocation[0])
        print("Agent",agent1.name,"is holding one piece", agent1.totalallocation[0])
        print("This piece is valued at",agent1.totalvalue, " by this agent")
    else:
        agent1.totalvalue = vop(agent1, agent1.totalallocation[0]) + vop(agent1, agent1.totalallocation[1])                   
        print("Agent",agent1.name,"is holding two pieces", agent1.totalallocation)
        print("These piece are valued at",agent1.totalvalue, " by this agent")  
      
    if agent2.totalallocation[1] == 0:
        agent2.totalvalue = vop(agent2, agent2.totalallocation[0])        
        print("Agent",agent2.name,"is holding one piece", agent2.totalallocation[0])
        print("This piece is valued at",agent2.totalvalue, " by this agent")
    else:
        agent2.totalvalue = vop(agent2, agent2.totalallocation[0]) + vop(agent2, agent2.totalallocation[1])         
        print("Agent",agent2.name,"is holding two pieces", agent2.totalallocation)
        print("These piece are valued at",agent2.totalvalue, " by this agent")  
            
    if agent3.totalallocation[1] == 0:
        agent3.totalvalue = vop(agent3, agent3.totalallocation[0])        
        print("Agent",agent3.name,"is holding one piece", agent3.totalallocation[0])
        print("This piece is valued at",agent3.totalvalue, " by this agent")
    else:
        agent3.totalvalue = vop(agent3, agent3.totalallocation[0]) + vop(agent3, agent3.totalallocation[1])         
        print("Agent",agent3.name,"is holding two pieces", agent3.totalallocation)
        print("These piece are valued at",agent3.totalvalue, " by this agent")  



#This function attributes each agent with a total value
#Same as previous function but without printing
def totalvaluecalc(agent1, agent2, agent3):
    if agent1.totalallocation[1] == 0:
        agent1.totalvalue = vop(agent1, agent1.totalallocation[0])
    else:
        agent1.totalvalue = vop(agent1, agent1.totalallocation[0]) + vop(agent1, agent1.totalallocation[1])                   
    if agent2.totalallocation[1] == 0:
        agent2.totalvalue = vop(agent2, agent2.totalallocation[0])        
    else:
        agent2.totalvalue = vop(agent2, agent2.totalallocation[0]) + vop(agent2, agent2.totalallocation[1])           
    if agent3.totalallocation[1] == 0:
        agent3.totalvalue = vop(agent3, agent3.totalallocation[0])        
    else:
        agent3.totalvalue = vop(agent3, agent3.totalallocation[0]) + vop(agent3, agent3.totalallocation[1])         

            
#This function returns a 1 if an agent is envious of another, and 0 otherwise
def compare(agent1, agent2):
    if (trimmingspiece(agent1) == 0) and (trimmingspiece(agent2) == 0):
        if (agent1.totalvalue < vop(agent1, agent2.allocation) or (agent2.totalvalue < vop(agent2, agent1.allocation))):
            return 1
        else:
            return 0
    elif((trimmingspiece(agent1) == 1) and (trimmingspiece(agent2) == 1)):
        if (agent1.totalvalue < (vop(agent1, agent2.allocation)+vop(agent1, agent2.trim)) or (agent2.totalvalue < vop(agent2, agent1.allocation)+vop(agent2, agent1.trim))):
            return 1
        else:
            return 0
    elif ((trimmingspiece(agent1) == 0) and (trimmingspiece(agent2) == 1)):
        if(agent1.totalvalue < (vop(agent1, agent2.allocation)+vop(agent1, agent2.trim)) or (agent2.totalvalue < vop(agent2, agent1.allocation))):
            return 1
        else:
            return 0
    elif ((trimmingspiece(agent1) == 1) and (trimmingspiece(agent2) == 0)):
        if(agent1.totalvalue < vop(agent1, agent2.allocation) or (agent2.totalvalue < vop(agent2, agent1.allocation)+vop(agent2, agent1.trim))):
            return 1
        else:
            return 0                                                     
                                                                   
                                                                   
                                                                   
                                                                                                                          
#This function determine if the allocation was envy free or not and prints the result
#We simply use a compare function to compare two agents and compare all possible pairs
#A returned value of 1 means it was envy free, 0 means it was not
def envyfreenesschecker(agent1, agent2, agent3):
    a = compare(agent1, agent2)
    b = compare(agent1, agent3)
    c = compare(agent2, agent3)
    if((a+b+c) >= 1):
        return 0
    else:
        return 1
    
#This function prints whether an allocation was envy free or not   
def envyfreeprinter(agent1, agent2, agent3):
    if envyfreenesschecker(agent1, agent2, agent3) == 0:
        print("This allocation was not envy free")
    else:
        print("This allocation was envy free")
    
    
    
    

In [2]:
#Determine how many atoms we want
atomsexample = 100

#Generate some agents
agentA = Agent("A", generatevaluations(atomsexample))
agentB = Agent("B", generatevaluations(atomsexample))
agentC = Agent("C", generatevaluations(atomsexample))

#Ask agent A to cut the cake into 3 pieces
piece1, piece2, piece3 = initialcuts(agentA, atomsexample)


#Ask agent B to trim his most preferred piece to equal his second (as best he can)
trimmedpiece, piece2, piece3, trimmings = createtrim(agentB, piece1, piece2, piece3)


#Ask have agents choose pieces as outlined in the Selfridge Conway procedure (Not allocating the trimmings yet)
firstallocation(agentA, agentB, agentC, trimmedpiece, piece2, piece3)


#Determine which agent is holding the trimmed piece and out of agent B and C
#Then allow the other agent to divide the trimmings.
trim1, trim2, trim3 = trimcut(agentB, agentC, trimmings)


#Have agents select trim pieces 
trimallocation(agentA, agentB, agentC, trim1, trim2, trim3)

#Determine the total allocation for each agent
determineta(agentA, agentB, agentC)

#Print infomation about the pieces they are holding and the total value an agent thinks he is holding
printallocation(agentA, agentB, agentC)

#Lastly will be a call to tell us if the allocation was envy free.
envyfreeprinter(agentA, agentB, agentC)


Agent A is holding one piece (37, 69)
This piece is valued at 0.33798449047627466  by this agent
Agent B is holding two pieces [(3, 36), (0, 2)]
These piece are valued at 0.3586744148058691  by this agent
Agent C is holding one piece (70, 99)
This piece is valued at 0.33692238015692244  by this agent
This allocation was not envy free


In [3]:
#This function will generate a single run through of the Selfridge Conway Procedure
#Returns a 1 if envy free, 0 if not.
def iteratingsrcon(atoms):
    agentA = Agent("A", generatevaluations(atoms))
    agentB = Agent("B", generatevaluations(atoms))
    agentC = Agent("C", generatevaluations(atoms))
    piece1, piece2, piece3 = initialcuts(agentA, atoms)
    trimmedpiece, piece2, piece3, trimmings = createtrim(agentB, piece1, piece2, piece3)
    firstallocation(agentA, agentB, agentC, trimmedpiece, piece2, piece3)
    trim1, trim2, trim3 = trimcut(agentB, agentC, trimmings)
    trimallocation(agentA, agentB, agentC, trim1, trim2, trim3)
    determineta(agentA, agentB, agentC)
    totalvaluecalc(agentA, agentB, agentC)
    return envyfreenesschecker(agentA, agentB, agentC)
    

In [4]:
#Iterate through 1000 Selfrdige Conway Procedures and determine number of envy free iterations
#Here you can edit the number of iterations or atoms
iterations = 1000
numberofatoms = 1000
counter = 0

for m in range(1, iterations+1, 1):
    counter = counter + iteratingsrcon(numberofatoms)
percentage = round(((counter/iterations)*100), 2)

print("With", numberofatoms, "atoms and", iterations, "iterations, we get that a total of",counter,"(",percentage,"%)iterations are envy free.")



With 1000 atoms and 1000 iterations, we get that a total of 440 ( 44.0 %)iterations are envy free.
